In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datetime import datetime
from collections import defaultdict
from matplotlib import pyplot as plt

In [2]:
#data = 'tokenized_answers.xlsx'
#data = 'tokenized_blogs.xlsx'
#data = 'tokenized_emails.xlsx'
data = 'tokenized_news.xlsx'
data_df = pd.read_excel(data)
X_df = data_df[['pos', 'text']]
y_df = data_df['score']
X_train_df, X_test_df, y_train_df, y_test_df =  train_test_split(X_df, y_df, test_size = 0.20, random_state=15)
X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(X_train_df, y_train_df, test_size=0.20, random_state=10) 

X_train_index = X_train_df.index
X_val_index = X_val_df.index
X_test_index = X_test_df.index
y_train_index = y_train_df.index
y_val_index = y_val_df.index
y_test_index = y_test_df.index

X_train_df = X_train_df.reset_index(drop=True)
X_val_df = X_val_df.reset_index(drop=True)
X_test_df = X_test_df.reset_index(drop=True)
y_train_np = y_train_df.values
y_val_np = y_val_df.values
y_test_np = y_test_df.values

pos_train_df = X_train_df[['pos']]
pos_val_df = X_val_df[['pos']]
pos_test_df = X_test_df[['pos']]

text_train_df = X_train_df[['text']]
text_train_df.text = text_train_df.text.astype(str)
text_val_df = X_val_df[['text']]
text_val_df.text = text_val_df.text.astype(str)
text_test_df = X_test_df[['text']]
text_test_df.text = text_test_df.text.astype(str)

/tmp/ipykernel_24864/1571134795.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_train_df.text = text_train_df.text.astype(str)
/tmp/ipykernel_24864/1571134795.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_val_df.text = text_val_df.text.astype(str)
/tmp/ipykernel_24864/1571134795.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [3]:
#embedding_npy = '../data/formality/bert_embedding_answers.npy'
#embedding_npy = '../data/formality/bert_embedding_blogs.npy'
#embedding_npy = '../data/formality/bert_embedding_emails.npy'
embedding_npy = '../data/formality/bert_embedding_news.npy'
X_np = np.load(embedding_npy,allow_pickle=True)
X_np_padded = tf.keras.preprocessing.sequence.pad_sequences(X_np,
                                                             padding='post')

2022-03-23 15:57:39.121811: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 15:57:41.881968: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11089 MB memory:  -> device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:03:00.0, compute capability: 5.2


In [4]:
bert_encoding_train_np = X_np_padded[X_train_index]
bert_encoding_val_np = X_np_padded[X_val_index]
bert_encoding_test_np = X_np_padded[X_test_index]

In [5]:
# Create pos vocabulary, encode pos data
pos_vocab = defaultdict(int)
pos_train_encode = []
pos_val_encode = []
pos_test_encode = []

for i in range(len(pos_train_df)):
    pos_tokens = pos_train_df['pos'][i]
    pos_tokens = eval(pos_tokens)
    encode = []
    
    for pos in pos_tokens:
        if pos not in pos_vocab:
            pos_vocab_size = len(pos_vocab)
            pos_vocab[pos] = pos_vocab_size + 1
        
        encode.append(pos_vocab[pos])
        
    pos_train_encode.append(encode)
    
for i in range(len(pos_val_df)):
    pos_tokens = pos_val_df['pos'][i]
    pos_tokens = eval(pos_tokens)
    encode = []
    
    for pos in pos_tokens:
        if pos not in pos_vocab:
            pos_vocab_size = len(pos_vocab)
            pos_vocab[pos] = pos_vocab_size + 1
        
        encode.append(pos_vocab[pos])
        
    pos_val_encode.append(encode)
    
for i in range(len(pos_test_df)):
    pos_tokens = pos_test_df['pos'][i]
    pos_tokens = eval(pos_tokens)
    encode = []
    
    for pos in pos_tokens:
        if pos not in pos_vocab:
            pos_vocab_size = len(pos_vocab)
            pos_vocab[pos] = pos_vocab_size + 1
        
        encode.append(pos_vocab[pos])
        
    pos_test_encode.append(encode)
    
# Pad pos encode
max_pos_timestep = 0
for i in range(len(pos_train_encode)):
    arr = pos_train_encode[i]
    l = len(arr)
    max_pos_timestep = max(max_pos_timestep, l)
    
for i in range(len(pos_val_encode)):
    arr = pos_val_encode[i]
    l = len(arr)
    max_pos_timestep = max(max_pos_timestep, l)
    
for i in range(len(pos_test_encode)):
    arr = pos_test_encode[i]
    l = len(arr)
    max_pos_timestep = max(max_pos_timestep, l)
padded_pos_train_encode = tf.keras.preprocessing.sequence.pad_sequences(pos_train_encode,
                                                                        maxlen=max_pos_timestep,
                                                                        padding='post')
padded_pos_val_encode = tf.keras.preprocessing.sequence.pad_sequences(pos_val_encode,
                                                                        maxlen=max_pos_timestep,
                                                                        padding='post')
padded_pos_test_encode = tf.keras.preprocessing.sequence.pad_sequences(pos_test_encode,
                                                                       maxlen=max_pos_timestep,
                                                                       padding='post')

In [6]:
epochs = 200
batch = 32
lr = 3e-5
loss = tf.keras.losses.MeanSquaredError()
metrics = tf.metrics.MeanSquaredError()

In [16]:
#pos model
pos_vocab_size = len(pos_vocab) + 1
pos_timesteps = padded_pos_train_encode.shape[-1]
features  = 256
pos_model = keras.Sequential()
pos_model.add(layers.Embedding(pos_vocab_size, features, input_length=pos_timesteps, mask_zero=True))
pos_model.add(layers.LSTM(features))


#text model
text_timesteps =  bert_encoding_train_np.shape[1]
text_features = bert_encoding_train_np.shape[2]
text_model = keras.Sequential()
text_model.add(layers.Masking(mask_value=0.,
                        input_shape=(text_timesteps, text_features)))
text_model.add(layers.LSTM(text_features))

model_concat = concatenate([pos_model.output, text_model.output], axis=-1)
dropout_1 = layers.Dropout(0.1)(model_concat)
dense_1 = Dense(512, activation='relu')(dropout_1)
dropout_2 = layers.Dropout(0.1)(dense_1)
dense_2 = Dense(256, activation='relu')(dropout_2)
dropout_3 = layers.Dropout(0.1)(dense_2)
dense_3 = Dense(128, activation='relu')(dropout_3)
dropout_4 = layers.Dropout(0.1)(dense_3)
dense_4 = Dense(64, activation='relu')(dropout_4)
dropout_5 = layers.Dropout(0.1)(dense_4)
dense_5 = Dense(32, activation='relu')(dropout_5)
output = Dense(1, activation='linear')(dense_5)
model = Model(inputs=[pos_model.input, text_model.input], outputs=output)
model.compile(optimizer=keras.optimizers.Adam(lr),
              loss=loss,
              metrics=metrics)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=10,
                                                  restore_best_weights=True)
callbacks = [early_stopping]
print(datetime.now())
history = model.fit([padded_pos_train_encode, bert_encoding_train_np], 
                    y_train_np, 
                    validation_data=([padded_pos_val_encode, bert_encoding_val_np], y_val_np),
                    epochs=epochs,
                    callbacks=callbacks)
print(datetime.now())

2022-03-23 16:06:12.314205
Epoch 1/200
56/56 [==============================] - 9s 73ms/step - loss: 0.7611 - mean_squared_error: 0.7225 - val_loss: 0.6905 - val_mean_squared_error: 0.6905
Epoch 2/200
56/56 [==============================] - 2s 40ms/step - loss: 0.6413 - mean_squared_error: 0.6413 - val_loss: 0.6188 - val_mean_squared_error: 0.6188
Epoch 3/200
56/56 [==============================] - 2s 39ms/step - loss: 0.5948 - mean_squared_error: 0.5948 - val_loss: 0.5767 - val_mean_squared_error: 0.5767
Epoch 4/200
56/56 [==============================] - 2s 39ms/step - loss: 0.5610 - mean_squared_error: 0.5610 - val_loss: 0.5311 - val_mean_squared_error: 0.5311
Epoch 5/200
56/56 [==============================] - 2s 40ms/step - loss: 0.5367 - mean_squared_error: 0.5367 - val_loss: 0.5344 - val_mean_squared_error: 0.5344
Epoch 6/200
56/56 [==============================] - 2s 39ms/step - loss: 0.5118 - mean_squared_error: 0.5118 - val_loss: 0.5240 - val_mean_squared_error: 0.5240
E

In [17]:
train_results = model.predict([padded_pos_train_encode, bert_encoding_train_np])
train_mse = mean_squared_error(train_results, y_train_np)
train_spearman_r = stats.spearmanr(train_results, y_train_np)
print(f"Train mse: {train_mse}")
print(f"Train spearman r: {train_spearman_r}")

Train mse: 0.43116471867401085
Train spearman r: SpearmanrResult(correlation=0.6019932702946851, pvalue=1.362101244181293e-175)


In [18]:
test_results = model.predict([padded_pos_test_encode, bert_encoding_test_np])
test_mse = mean_squared_error(test_results, y_test_np)
test_spearman_r = stats.spearmanr(test_results, y_test_np)
print(f"Test mse: {test_mse}")
print(f"Test spearman r: {test_spearman_r}")

Test mse: 0.5023148336431259
Test spearman r: SpearmanrResult(correlation=0.4924518938703908, pvalue=3.04438705731779e-35)
